In [2]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 603.2 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 778.4/778.4 kB 5.2 MB/s eta 0:00:0000:0100:01


In [3]:
!pip install -U ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 1.5 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.0/215.0 kB 4.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 23.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.6
    Uninstalling widgetsnbextension-3.6.6:
      Successfully uninstalled widgetsnbextension-3.6.6
  Attempting uninstall: jupyterlab-widgets
    Found existing installation: jupyterlab-widgets 3.0.9
    Uninstalling jupyterlab-widgets-3.0.9:
      Successfully uninstalled jupyterlab-widgets-3.0.9
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


In [4]:
from ultralytics import YOLO
import os

#### Trainning the model

In [ ]:
results = model.train(
    data= '/kaggle/input/80-10-10-dataset/data.yaml',
    epochs=100,
    imgsz=640,
    patience=50,
    device=0,
    batch=64,
    optimizer= 'SGD',
    momentum=0.95
)

#### Evalution

In [5]:
model = YOLO("/kaggle/input/best-model/best_80_10_10.pt")  

metrics = model.val()  
metrics.box.map  
metrics.box.map50  
metrics.box.map75  
metrics.box.maps  

Ultralytics YOLOv8.2.22 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3006233 parameters, 0 gradients, 8.1 GFLOPs


100%|██████████| 755k/755k [00:00<00:00, 15.3MB/s]
val: Scanning /kaggle/input/80-10-10-dataset/valid/labels... 1016 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1016/1016 [00:03<00:00, 293.06it/s]


val: WARNING ⚠️ Cache directory /kaggle/input/80-10-10-dataset/valid is not writeable, cache not saved.


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 64/64 [00:15<00:00,  4.04it/s]


                   all       1016       1872      0.897      0.817      0.901      0.562
              Drowning       1016       1107      0.886      0.875      0.934      0.606
              Swimming       1016        707      0.856      0.851      0.886      0.508
          out of water       1016         58       0.95      0.724      0.885      0.571
Speed: 0.4ms preprocess, 3.7ms inference, 0.0ms loss, 4.7ms postprocess per image
Results saved to runs/detect/val


array([    0.60647,     0.50794,     0.57074])

#### hyperparameter-tuning

- in our case SGD as optimizer is better that AdamW


In [9]:
model = YOLO("/kaggle/input/best-model/best_80_10_10.pt") 

- lr0 = [0.0005, **0.001**, 0.005]
- lrf = [0.01, **0.005**]
- batch = [16, **32**, 64]
- momentum = [0.85, **0.9**, 0.95]
- warmup_momentum = [**0.5**]
- warmup_bias_lr = [**0.05**, 0.1, 0.2]
- weight_decay = [0.0001, **0.0005**, 0.001]
- patience = [**30**, 50]
- imgsz = [**640**, 800, 1024]


In [ ]:
results = model.train(
    data='/kaggle/input/80-10-10-dataset/data.yaml',
    epochs=50,
    imgsz=640,
    patience=30,  
    device=0,
    batch=32,  
    optimizer='SGD',  
    lr0=0.001,  
    lrf=0.005,
    momentum=0.9,
    warmup_momentum=0.5,
    warmup_bias_lr=0.05,
    weight_decay=0.0005,  
)


Ultralytics YOLOv8.2.22 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/kaggle/input/best-model/best_80_10_10.pt, data=/kaggle/input/80-10-10-dataset/data.yaml, epochs=50, time=None, patience=30, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show

wandb: Currently logged in as: ali-vexll0 (t5-sadia). Use `wandb login --relogin` to force relogin


Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /kaggle/input/80-10-10-dataset/train/labels... 7981 images, 0 backgrounds, 0 corrupt: 100%|██████████| 7981/7981 [00:16<00:00, 497.37it/s]


train: WARNING ⚠️ Cache directory /kaggle/input/80-10-10-dataset/train is not writeable, cache not saved.
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      40/50      4.42G      1.077     0.7502      1.259         52        640: 100%|██████████| 250/250 [01:17<00:00,  3.23it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:06<00:00,  2.29it/s]

                   all       1016       1872      0.885      0.821      0.897      0.564


Closing dataloader mosaic
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      41/50      4.77G      1.123     0.6478      1.304         57        640:  97%|█████████▋| 243/250 [01:15<00:02,  3.14it/s]

#### Testing the model

In [ ]:
from ultralytics.utils.plotting import Annotator
import cv2

In [ ]:
model = YOLO("/kaggle/input/best-model/best_80_10_10.pt") 
video_path = 'drowing_video.mp4'
cap = cv2.VideoCapture(video_path)

In [ ]:
while True:
    ret, frame = cap.read()
    if not ret:
        break
        
    results = model(frame)
    
    for r in results:
        annotator = Annotator(frame)
        
        boxes = r.boxes
        for box in boxes:
            b = box.xyxy[0]
            c = box.cls
            annotator.box_label(b, model.names[int(c)])
    
    frame = annotator.result()
    cv2.imshow("Object Detection", frame)
    
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()